In [56]:
import numpy as np
import tensorflow as tf
import glob
import pandas as pd
import re
import os
import time
# Load TFLite model and allocate tensors.

interpreter = tf.lite.Interpreter(model_path="/Users/tousif/LightHART-tf/exp/smartfallmm/test/models/transformer_scratch.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']

In [57]:
root_dir = '/Users/tousif/LightHART-tf/'
data_files = os.path.join(root_dir, 'datasets/smartfallmm/online_data/*/*.csv')
path_list = glob.glob(data_files)

In [59]:
import pandas
from queue import Queue

tp = 0
tn = 0
fp = 0
fn = 0

for path in path_list[:2]:
  print(path)
  label = 1 if 'fall' in path.split('/')[-1] else 0
  print(label)
  df = pd.read_csv(path)
  df1 = df[['Acc_x','Acc_y','Acc_z']]
  i = 0
  fall_count = 0
  average = 20
  queue = []
  times = []
  while i < df1.shape[0] and i+128 <= df1.shape[0]:
      arr = np.array(df1[i:i+128], dtype=np.float32)
      input_data = []
      input_data.append(arr)
      interpreter.set_tensor(input_details[0]['index'], input_data)
      start_time = time.time()
      interpreter.invoke()
      inference_time = time.time() - start_time
      print(inference_time*1000)
      output_data = interpreter.get_tensor(output_details[0]['index'])
      output = tf.argmax(tf.nn.softmax(output_data,axis=-1), axis =1)

      queue.append(output.numpy())
      if len(queue) == average:
        output = 0.0
        for num in queue:
          output=output+num
        output = output/average

        # print(output)
        if output == 1:
          fall_count = fall_count+1
        queue.pop(0)
        i += 1
      
  if fall_count > 0 and label == 1 : 
    tp += 1

  elif fall_count > 0 and label == 0:
    fp += 1
  
  elif fall_count == 0 and label == 0 :
    tn += 1
  
  else:
    fn += 0



print('fall count is: ')
print('True Positive: {}'.format(tp))
print('True Negative: {}'.format(tn))
print('False Positive: {}'.format(fp))
print('False Negative: {}'.format(fn))

/Users/tousif/LightHART-tf/datasets/smartfallmm/online_data/awatif/sit5.csv
0
1.4681816101074219
0.8168220520019531
0.7212162017822266
0.637054443359375
0.6198883056640625
0.5412101745605469
0.8766651153564453
0.5152225494384766
0.6346702575683594
0.5838871002197266
0.5702972412109375
0.4971027374267578
0.4761219024658203
0.5049705505371094
0.47397613525390625
0.5691051483154297
0.5030632019042969
0.45418739318847656
0.5102157592773438
0.5497932434082031
0.48804283142089844
0.47898292541503906
0.4749298095703125
0.45180320739746094
0.4990100860595703
0.42700767517089844
0.5400180816650391
0.4520416259765625
0.4208087921142578
0.4239082336425781
0.42510032653808594
0.44608116149902344
0.4570484161376953
0.5199909210205078
0.4818439483642578
0.45800209045410156
0.48804283142089844
0.4470348358154297
0.41604042053222656
0.4150867462158203
0.4279613494873047
0.4191398620605469
0.4138946533203125
0.43082237243652344
0.43010711669921875
0.4222393035888672
0.41413307189941406
0.42915344238281

In [51]:
(24+5)/(24+5+35)


0.453125

In [ ]:
'kd'
True Positive: 24
True Negative: 25
False Positive: 15
False Negative: 0

In [ ]:
True Positive: 25
True Negative: 5
False Positive: 30
False Negative: 5